In [84]:
import numpy as np
import librosa, librosa.util
from sklearn.decomposition import NMF
from scipy.io.wavfile import write
from librosa.core import icqt
import librosa.display
import IPython.display

In [93]:
y ,sr = librosa.load('.`/resource/2-poly.wav')
C = librosa.cqt(y, sr=sr, hop_length=64, n_bins=84, bins_per_octave=12)
C_n = np.abs(C)

# Use Examples Ref  
```python
wav = icqt(C, sr=sr, hop_length=64, res_type='fft')
write('after.wav', sr, wav)
IPython.display.Audio(data=y, rate=sr)
```

# Todos  
- [x] Check **librosa** official example of [harmonic/percussive separation](https://nbviewer.jupyter.org/github/librosa/librosa/blob/master/examples/LibROSA%20audio%20effects%20and%20playback.ipynb)  
- [ ] Check possible solutions to the bottleneck of icqt problem  
    - [ ] Manully add virtual part to the matrix  
    > Depending on how it would affect the icqt  
    - [ ] Try other resample models  

In [111]:
def source_separation(y, sr):
    n_components = 2
    nmf_model = NMF(n_components=n_components, init='random', max_iter=500, alpha=0, l1_ratio=0.0)
    
    S_full, phase = librosa.magphase(C)
    print(type(S_full[0][0]), type(phase[0][0]))
    
    W = nmf_model.fit_transform(S_full)
    H = nmf_model.components_
    print(nmf_model.reconstruction_err_)
    
    n_W = []
    n_H = []
    n_S = []
    n_C = []
    n_wav = []
    for i in range(0, n_components):
        w = W[:,i:i+1]
        h = H[i:i+1, :]
        s = np.multiply(w, h)
        c = s * phase
        wav = icqt(c, sr=sr, hop_length=64)

        n_W.append(w)
        n_H.append(h)
        n_S.append(s)
        n_C.append(c)
        n_wav.append(wav)
    
    return n_wav

In [ ]:
n_y = y.reshape((16, 5776))
n_wav = source_separation(n_y[1])

IPython.display.Audio(data=n_wav[0], rate=sr)
IPython.display.Audio(data=n_wav[1], rate=sr)

# Todos
[librosa.beat.tempo](https://librosa.github.io/librosa/generated/librosa.beat.tempo.html#librosa.beat.tempo) could be used to detect and estimate tempo.  
[librosa.onset.onset_detect](https://librosa.github.io/librosa/generated/librosa.onset.onset_detect.html#librosa.onset.onset_detect) could be used to detect onset.  
[librosa.decompose.decompose](https://librosa.github.io/librosa/generated/librosa.decompose.decompose.html#librosa.decompose.decompose) decompose a feature matrix with **scikit-learn** as well.  